In [ ]:
import pandas as pd
from typing import Tuple, NamedTuple, List
from collections import namedtuple
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import trange, tqdm
from pandas.tseries.offsets import DateOffset
import datetime

In [ ]:
COMMITMENT_OF_TRADERS_FUTURES_AND_OPTIONS_FILE_PATH = '../data/raw/nasdaq_data_link/commitment_of_trade/CFTC-057642_FO_ALL_OI.csv'
CONTRACT_INTRADAY_SLIDING_OPEN_FILE_PATH = '../data/processed/futures_contracts/contract_open_enriched_sliding_open.csv'
CONTRACT_INTRADAY_TRUE_OPEN_FILE_PATH = '../data/processed/futures_contracts/contract_open_enriched_true_open.csv'

In [ ]:
def intraday_open_csv_to_df(filename) -> pd.DataFrame:
    csv_as_df = pd.read_csv(
        filename,
        parse_dates=['DateTime'], 
        usecols=[
          'Symbol','DateTime','Open Minutes Offset','Open','High','Low','Close',
            'Volume','Price Change From Intraday Open','Expiration Date','DTE'
          ]
    )
    return csv_as_df

In [ ]:
def cot_csv_to_df(filename) -> pd.DataFrame:
  '''Convert the commitment of traders report to a dataframe with the relevant columns'''
  csv_as_df = pd.read_csv(
      filename,
      parse_dates=['Date'], 
      usecols=['Date', 'Producer/Merchant/Processor/User Shorts - % of OI']
  )
  return csv_as_df

In [ ]:
def filter_and_split_cot_shorts_around_median(
  cot_df: pd.DataFrame,
  median_pmpu_shorts_percentage: float
) -> NamedTuple:
  cot_pmpu_shorts_above_median_df = cot_df[cot_df['Producer/Merchant/Processor/User Shorts - % of OI'] >= median_pmpu_shorts_percentage].copy().reset_index(drop=True)
  cot_pmpu_shorts_below_median_df = cot_df[cot_df['Producer/Merchant/Processor/User Shorts - % of OI'] < median_pmpu_shorts_percentage].copy().reset_index(drop=True)
  pmpu_shorts_split_by_median = namedtuple('pmpu_around_median', ['above_median_df', 'below_median_df'])
  return pmpu_shorts_split_by_median(cot_pmpu_shorts_above_median_df, cot_pmpu_shorts_below_median_df)

In [ ]:
def is_date_inside_range_of_dataframe(a_date: pd.Timestamp, cot_pmpu_shorts_df: pd.DataFrame) -> bool:
  rows_with_a_date_inside = cot_pmpu_shorts_df[(cot_pmpu_shorts_df['Begin Apply Date Range'] <= a_date) & (cot_pmpu_shorts_df['End Apply Date Range'] >= a_date)]
  contains_rows_with_date_inside = rows_with_a_date_inside['Date'].any()
  if contains_rows_with_date_inside:
    print('True')
  return contains_rows_with_date_inside

In [ ]:
def date_of_preceding_tuesday(a_date: pd.Timestamp) -> pd.Timestamp:
  day_of_week = a_date.date().weekday()
  match day_of_week:
    case 0: # Monday
      days_since_preceding_tuesday = 6
    case 1: # Tuesday
      days_since_preceding_tuesday = 7
    case 2: # Wednesday
      days_since_preceding_tuesday = 8
    case 3: # Thursday
      days_since_preceding_tuesday = 9
    case 4: # Friday
      days_since_preceding_tuesday = 10
    case 5: # Saturday
      days_since_preceding_tuesday = 11
    case 6: # Sunday
      days_since_preceding_tuesday = 12
  preceding_tuesday_date = a_date - datetime.timedelta(days=days_since_preceding_tuesday)
  return preceding_tuesday_date.date()

In [ ]:
def split_intraday_activity_by_cot_median(
  cot_pmpu_shorts_above_median_df: pd.DataFrame,
  cot_pmpu_shorts_below_median_df: pd.DataFrame,
  intraday_open_df: pd.DataFrame
) -> NamedTuple:
  dates_pmpu_shorts_above_median = cot_pmpu_shorts_above_median_df['Date'].dt.date.drop_duplicates()
  dates_pmpu_shorts_below_median = cot_pmpu_shorts_below_median_df['Date'].dt.date.drop_duplicates()
  open_bars_where_cot_pmpu_shorts_above_median_df = intraday_open_df[intraday_open_df['Date Of Preceding Tuesday'].isin(dates_pmpu_shorts_above_median)]
  open_bars_where_cot_pmpu_shorts_below_median_df = intraday_open_df[intraday_open_df['Date Of Preceding Tuesday'].isin(dates_pmpu_shorts_below_median)]
  intraday_minute_bars_split = namedtuple('intraday_minute_bars_split', ['above_median', 'below_median'])
  return intraday_minute_bars_split(open_bars_where_cot_pmpu_shorts_above_median_df, open_bars_where_cot_pmpu_shorts_below_median_df)

In [ ]:
def calculate_average_intraday_price_change_grouped_by_open_minutes_offset(intraday_minute_bars_df: pd.DataFrame) -> pd.DataFrame:
  '''
  Group the intraday minute bars by their Open Minutes Offset and calculate the mean for each minute. Return all that as a single dataframe
  '''
  intraday_above_median_pmpu_shorts_df = intraday_minute_bars_df.above_median.groupby('Open Minutes Offset', as_index=False)['Price Change From Intraday Open'].mean()
  intraday_below_median_pmpu_shorts_df = intraday_minute_bars_df.below_median.groupby('Open Minutes Offset', as_index=False)['Price Change From Intraday Open'].mean()
  to_return_df = pd.DataFrame({
    'Open Minutes Offset': intraday_above_median_pmpu_shorts_df['Open Minutes Offset'],
    'Avg Intraday Price Change When P/M/P/U Short Interest Is Above Median': intraday_above_median_pmpu_shorts_df['Price Change From Intraday Open'],
    'Avg Intraday Price Change When P/M/P/U Short Interest Is Below Median': intraday_below_median_pmpu_shorts_df['Price Change From Intraday Open']
  })
  return to_return_df

In [ ]:
def generate_figure(intraday_price_changes_split_df: pd.DataFrame, fig_title: str, median_pmpu_shorts_percentage: float) -> go.Figure:
  fig = go.Figure()
  fig.add_trace(go.Scatter(
    x=intraday_price_changes_split_df['Open Minutes Offset'],
    y=intraday_price_changes_split_df['Avg Intraday Price Change When P/M/P/U Short Interest Is Above Median'],
    mode='lines+markers',
    name=f"When P/M/P/U Short Interest Pct Above Median={median_pmpu_shorts_percentage}")
    )
  fig.add_trace(go.Scatter(
    x=intraday_price_changes_split_df['Open Minutes Offset'],
    y=intraday_price_changes_split_df['Avg Intraday Price Change When P/M/P/U Short Interest Is Below Median'],
    mode='lines+markers',
    name=f"When P/M/P/U Short Interest Pct Below Median={median_pmpu_shorts_percentage}")
    )
  fig.update_xaxes(title_text='Minutes After Open')
  fig.update_yaxes(title_text='Avg Price Change From Open')
  fig.update_layout(title_text=fig_title)
  return fig

In [ ]:
def calc_begin_cot_apply_date_range(report_observation_date: pd.Timestamp) -> Tuple[pd.Timestamp, pd.Timestamp]:
  begin_apply_date = report_observation_date + DateOffset(days=6)
  return begin_apply_date


In [ ]:
def calc_end_cot_apply_date_range(report_observation_date: pd.Timestamp) -> Tuple[pd.Timestamp, pd.Timestamp]:
  end_apply_date = report_observation_date + DateOffset(days=12)
  return end_apply_date

In [ ]:
def enrich_with_apply_dates(cot_df: pd.DataFrame) -> pd.DataFrame:
  begin_apply_date_series = cot_df['Date'].apply(calc_begin_cot_apply_date_range)
  end_apply_date_series = cot_df['Date'].apply(calc_end_cot_apply_date_range)
  enriched_df = cot_df.copy()
  enriched_df['Begin Apply Date Range'] = begin_apply_date_series
  enriched_df['End Apply Date Range'] = end_apply_date_series
  return enriched_df

In [ ]:
intraday_sliding_open_df = intraday_open_csv_to_df(CONTRACT_INTRADAY_SLIDING_OPEN_FILE_PATH)
intraday_true_open_df = intraday_open_csv_to_df(CONTRACT_INTRADAY_TRUE_OPEN_FILE_PATH)
cot_df = cot_csv_to_df(COMMITMENT_OF_TRADERS_FUTURES_AND_OPTIONS_FILE_PATH)

In [ ]:
cot_df = enrich_with_apply_dates(cot_df)

In [ ]:
unique_symbols = list(intraday_true_open_df.Symbol.unique())

In [ ]:
median_pmpu_shorts_percentage = cot_df['Producer/Merchant/Processor/User Shorts - % of OI'].median()

In [ ]:
pmpu_shorts_split_by_median = filter_and_split_cot_shorts_around_median(cot_df, median_pmpu_shorts_percentage)

In [ ]:
cot_df['Weekday'] = cot_df['Date'].apply(lambda x: x.date().weekday())

In [ ]:
intraday_true_open_df['Date Of Preceding Tuesday'] = intraday_true_open_df['DateTime'].apply(date_of_preceding_tuesday)
intraday_sliding_open_df['Date Of Preceding Tuesday'] = intraday_sliding_open_df['DateTime'].apply(date_of_preceding_tuesday)

In [ ]:
intraday_minute_bars_true_open_split = split_intraday_activity_by_cot_median(
  cot_pmpu_shorts_above_median_df=pmpu_shorts_split_by_median.above_median_df,
  cot_pmpu_shorts_below_median_df=pmpu_shorts_split_by_median.below_median_df,
  intraday_open_df=intraday_true_open_df
)
intraday_minute_bars_sliding_open_split = split_intraday_activity_by_cot_median(
  cot_pmpu_shorts_above_median_df=pmpu_shorts_split_by_median.above_median_df,
  cot_pmpu_shorts_below_median_df=pmpu_shorts_split_by_median.below_median_df,
  intraday_open_df=intraday_sliding_open_df
)

In [ ]:
true_open_intraday_average_changes = calculate_average_intraday_price_change_grouped_by_open_minutes_offset(intraday_minute_bars_true_open_split)
sliding_open_intraday_average_changes = calculate_average_intraday_price_change_grouped_by_open_minutes_offset(intraday_minute_bars_sliding_open_split)

In [ ]:
fig1 = generate_figure(
  intraday_price_changes_split_df=true_open_intraday_average_changes,
  fig_title='True Open - Avg Intraday Open Split By P/M/P/U Short Interest',
  median_pmpu_shorts_percentage=median_pmpu_shorts_percentage
  )
fig2 = generate_figure(
  intraday_price_changes_split_df=sliding_open_intraday_average_changes,
  fig_title='Sliding Open - Avg Intraday Open Split By P/M/P/U Short Interest',
  median_pmpu_shorts_percentage=median_pmpu_shorts_percentage
)

Show the figures

In [ ]:
figures = [fig1, fig2]
for fig in figures:
  fig.show()